In [7]:
import maadsbml
# Uncomment IF using jupyter notebook
import nest_asyncio
# Uncomment IF using jupyter notebook
nest_asyncio.apply()

In [10]:
host='http://localhost'
port=5595

def hypertraining(host,port,filename,dependentvariable,removeoutliers,hasseasonality):
  #def hypertraining(host,port,filename,dependentvariable,removeoutliers=0,hasseasonality=0,summer='6,7,8',winter='11,12,1,2',shoulder='3,4,5,9,10',trainingpercentage=70,shuffle=0,deepanalysis=0,username='admin',timeout=1200,company='otics',password='123',email='support@otics.ca',usereverseproxy=0,microserviceid='',maadstoken='123',mode=0):

  #host,port,
  #filename= raw data file in csv format - Note this file is stored on your host machine the DOCKER container needs to be mapped to this volume using -v
  #dependentvariable= dependent variable name - this is the column name in the csv file
  # the file should have a Date column in the format Month/Day/Year
  #username= you can specify a username
  #mode=0
  #abort=0 set to 1 to abort the whole process, otherwise set to 0.  If Aborting, you should change port to 10000 to FORCE ABORT OF ANY RUNNING PROCESSES  
  #timeout=180 - you can modify this in seconds if your data file is large
  #company= change this to the name of your company
  #removeoutliers= specify 1 or 0, 1=remove outliers, 0 do not remove outliers,
  #hasseasonality= specify 1 or 0 to indicate date is affected by seasonaility - 1 = seasonality, 0 = no seasonality,
  #summer= specify the summer months ie. '6,7,8', or set to -1 for no summer
  #winter= specify winter months i.e. '11,12,1,2', or -1 for no winter
  #shoulder= specify shoulder months i.e. '3,4,5,9,10', or -1 for no shoulder season
  #trainingpercentage= specify training percentage i.e. 70, the value represents a percentage to split training and test
  #shuffle= specify 1 or 0 to shuffle the data, 1= shuffle, 0 = no shuffle
  #deepanalysis= specify 1 or 0, 1=deepanalysis, note this will run through deeper algorithms but will take longer, 0 = no deep analysis, this will
  #password='123', - leave as is
  #email='support@otics.ca', - leave as is
  #usereverseproxy=0, - leave as is
  #microserviceid='', leave as is
  #maadstoken='123' leave as is
  summer='6,7,8'
  winter='11,12,1,2'
 # shoulder='3,4,5,9,10'
  shoulder='-1'
  trainingpercentage=70
  shuffle=0
  res=maadsbml.hypertraining(host,port,filename,dependentvariable,removeoutliers,hasseasonality,summer,winter,shoulder,trainingpercentage,shuffle)
  print(res)


def hyperprediction(pkey,host,port,inputdata,username):
  
  res=maadsbml.hyperpredictions(pkey,inputdata,host,port,username)
  print(res)

def hyperpredictioncustom(pkey,host,port,inputdata,username,algoname,season):
  res=maadsbml.hyperpredictions(pkey,inputdata,host,port,username,algoname,season)
  print(res)
  
def algoinfo(pk):
   res=maadsbml.algodescription(host,port,pk)
   print(res)

def rundemo(demotype):
    # if demotype=1 then Regression will be run
    # if demotype=0 then Classification will be run
   res=maadsbml.rundemo(host,port,demotype)
   print(res)

def abort(host,port):
    # if demotype=1 then Regression will be run
    # if demotype=0 then Classification will be run
   res=maadsbml.abort(host,port)
   print(res)


# ############Function Commands
# Algoinfo
pk='admin_aesopowerdemand_csv'
algoinfo(pk)

#pk='admin_aesopowerdemandlogistic_csv'
#algoinfo(pk)

# ############Abort
#abort(host,10000)

# ############Rundemo
rundemo(1)

 ############Hypertraining
filename='aesopowerdemandlogistic.csv'
dependentvariable='AESO_Power_Demand_Label'

filename='aesopowerdemand.csv'
#filename='aesopowerdemandsm.csv'
dependentvariable='AESO_Power_Demand'
removeoutliers=1
hasseasonality=1
#hypertraining(host,port,filename,dependentvariable,removeoutliers,hasseasonality)


# ############Hyperpredictions
port=5495
pkey='admin_aesopowerdemand_csv'
inputdata='5/10/2010,-14.3,-32.0,-12.0'
#hyperprediction(pkey,host,port,inputdata,'admin')

#hyperpredictioncustom(pkey,host,port,inputdata,'admin','LogisticRegression','winter')

pkey='admin_aesopowerdemand_csv'
inputdata='5/10/2010,-14.3,-32.0,-12.0'
#hyperprediction(pkey,host,port,inputdata,'admin')

algo='BayesianRidge'
algo='simpleregression_reg'
season='summer'
#hyperpredictioncustom(pkey,host,port,inputdata,'admin',algo,season)



http://localhost:5595/?algoinfo=1&pkey=admin_aesopowerdemand_csv
{"RUN DATE": "2024/04/02", "RUN TIME": "2329", "ALGO KEY": "admin_aesopowerdemand_csv", "ALGO NAME": "TheilSenRegressor", "ALGO DESCRIPTION": "Theil-Sen Estimator (http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.TheilSenRegressor.html#sklearn.linear_model.TheilSenRegressor) - Theil-Sen Estimator: robust multivariate regression model.", "FORECAST ACCURACY": 0.594, "TRAINING SAMPLE SIZE": 897, "NUMBER OF INPUTS": 3, "SEASON": "N", "DEPENDENT VARIABLE": "AESO_Power_Demand", "TOTAL NUMBER OF ROWS": 1280, "OUTLIERS REMOVED": "0", "FIELDNAMES": "Date,AESO_Power_Demand,Calgary_Weather,Edmonton_Weather,FtMac_Weather", "COMPANY": "Alberta-Electric-System-Operator_AESO)", "EMAIL": "support@otics.ca", "USERNAME": "admin", "IS NORMALIZED?": "Y", "PDF REPORT": "/maads/agentfilesdocker/dist/maadsweb/pdfreports/admin_aesopowerdemand_csv.pdf", "TRAINING FILE": "/maads/agentfilesdocker/dist/maadsweb/csvuploads/aesopo

In [9]:

# ############Hyperpredictions
port=5495
pkey='admin_aesopowerdemand_csv'
inputdata='5/10/2010,-14.3,-32.0,-12.0'
hyperprediction(pkey,host,port,inputdata,'admin')

pkey='admin_aesopowerdemandlogistic_csv'
inputdata='5/10/2010,-14.3,-32.0,-12.0'
hyperprediction(pkey,host,port,inputdata,'admin')

{"hyperprediction":10164.472,"AlgoKey":"admin_aesopowerdemand_csv","Season":"allseason","Algorithm":"TheilSenRegressor","Dependent Variable":"AESO_Power_Demand","Fields":"Date,Calgary_Weather,Edmonton_Weather,FtMac_Weather","Trained Model Accuracy":"0.593805","Pickle Files":"/maads/agentfilesdocker/networks/Alberta-Electric-System-Operator_AESO)_ADMIN_AESOPOWERDEMAND_CSVALLSEASON_AG1_4_TheilSenRegressor_normal_897_ensemble_.pkl, /maads/agentfilesdocker/networks/Alberta-Electric-System-Operator_AESO)_ADMIN_AESOPOWERDEMAND_CSVALLSEASON_AG1_4_TheilSenRegressor_normal_897_ensemble_scalerx_.pkl, /maads/agentfilesdocker/networks/Alberta-Electric-System-Operator_AESO)_ADMIN_AESOPOWERDEMAND_CSVALLSEASON_AG1_4_TheilSenRegressor_normal_897_ensemble_scalery_.pkl","CreatedOn":"2024-04-03, 00:25:11","InputData":"5/10/2010,-14.3,-32.0,-12.0","MicroService":"OTICS_MAADS_AUTO-ML_HYPER-PREDICTION","Host":"localhost","Port":5495}
{"hyperprediction":[0.387,0.613],"AlgoKey":"admin_aesopowerdemandlogistic_